Introduction

In [ ]:
#load data

import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#to see graphs in colab or jupyter notebook, set plotly in offline.
import plotly
from plotly.graph_objs import Scatter, Layout
plotly.offline.init_notebook_mode()
#if you set this code and it doesnt work, upgrade jupyter or colab.
## there are 2 ways to use plotly. first is graph_objs. the other is plotly express.

import plotly.offline as py
py.init_notebook_mode(connected=1)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold

feature exploration, engineering and clearning

In [ ]:
#load in the train and test datasets
directory='/content/drive/MyDrive/주피터_대피소/kaggle&github/1.titanic/'


train=pd.read_csv(directory+'train.csv')
test=pd.read_csv(directory+'test.csv')


# Store out passenger ID for east access
PassengerId = test['PassengerId']



In [ ]:
test['Sex'].isnull().sum()

0

Feature engineering

In [ ]:
full_data=[train,test]


#1.length of the name

train['Name_len']=train['Name'].apply(len)
test['Name_len']=test['Name'].apply(len)

#2. Cabin
train['Has_Cabin']=train['Cabin'].apply(lambda x : 0 if type(x) == float else 1)
test['Has_Cabin']=test['Cabin'].apply(lambda x : 0 if type(x) == float else 1)
#before ensemble, he also set cabin 1 or 0. nvm.

#3. Family size and alone

for dataset in full_data :
  dataset['FamilySize']=dataset['SibSp']+dataset['Parch']+1

for dataset in full_data :
  dataset['IsAlone']=0
  dataset.loc[dataset['FamilySize']==1, 'IsAlone']=1


#4. Embarked
for dataset in full_data :
  dataset['Embarked'] = dataset['Embarked'].fillna('S')

#5. Fare
dataset['Fare']=dataset['Fare'].fillna(train['Fare'].mean()) #use mean or median. anything will be ok.

#6. Age
for dataset in full_data :
  age_avg=dataset['Age'].mean()
  age_std=dataset['Age'].std()
  age_null_count = dataset['Age'].isnull().sum()
  age_null_random_list= np.random.randint(age_avg- age_std , age_avg+age_std, size=age_null_count)
  # just make them inside the 1 std. cuz it's probability that all of null values in it is 96%.

  dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
  # nan values has probability that it fits in the std is 96%. 
  dataset['Age']=dataset['Age'].astype(int)

train['CategoricalAge']=pd.qcut(train['Age'],5)

#7. name

def get_title(name) :
  title_search=re.search('([A-Za-z]+)\.', name)              
  #or use str.extract. i think use extract is much economical than this.

  if title_search :
    return title_search.group(1)
  return ""

for dataset in full_data :
  dataset['Title']=dataset['Name'].apply(get_title)

for dataset in full_data : 
  dataset['Title']= dataset['Title'].replace(['Lady','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'], 'Rare')

  dataset['Title']=dataset['Title'].replace('Mlle','Miss')
  dataset['Title']=dataset['Title'].replace('Ms','Miss')
  dataset['Title']=dataset['Title'].replace('Mme','Mrs')



#9. mapping

#9-0. Sex

for dataset in full_data :
  dataset['Sex'] = dataset['Sex'].map({'female': 0, 'male': 1}).astype(int)


#9-1. title

title_mapping={'Mr' :0 ,'Miss' :1 , 'Mrs' :2 , 'Master' :3 , 'Rare' :4}
for dataset in full_data :
  dataset['Title']=dataset['Title'].map(title_mapping)
  dataset['Title']=dataset['Title'].fillna(4)

#9-2. embarked

dataset['Embarked'] = dataset['Embarked'].map({'S':0,'C':1,'Q':2,}).astype(int)

#9-3. Fare
## qcut 1,2,3,4로 끊을수있음?
dataset.loc[ dataset['Fare'] <= 7.91, 'Fare']=0
dataset.loc[ (7.91 < dataset['Fare']) & (dataset['Fare'] <= 14.454), 'Fare'] =1
dataset.loc[ (14.454 < dataset['Fare']) & (dataset['Fare']<= 31), 'Fare']=2
dataset.loc[ dataset['Fare'] >31, 'Fare']=3

dataset['Fare']=dataset['Fare'].astype(int)

###if we use def to cut fare

# def cut_fare(x) :
#   if x <=7.91 :
#     return 0
#   elif 7.91<x<=14.454 :
#     return 1
#   elif 14.454 <x<=31 :
#     return 2
#   else :
#     return 3

# dataset['Fare']=dataset['Fare'].map(lambda x : cut_fare(x))

#9-4. Age
dataset.loc[ dataset['Age'] <= 16, 'Age']=0
dataset.loc[  (16 < dataset['Age'])& (dataset['Age'] <= 32), 'Age'] =1
dataset.loc[ (32 < dataset['Age'])& (dataset['Age'] <= 48), 'Age']=2
dataset.loc[ (48 < dataset['Age'])& (dataset['Age'] <= 64), 'Age']=3
dataset.loc[ dataset['Fare'] >64, 'Age']=4

dataset['Age']=dataset['Age'].astype(int)



In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_len,Has_Cabin,FamilySize,IsAlone,CategoricalAge,Title
0,1,0,3,"Braund, Mr. Owen Harris",1,22,1,0,A/5 21171,7.2500,NaN,S,23,0,2,0,"(19.0, 25.0]",0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38,1,0,PC 17599,71.2833,C85,C,51,1,2,0,"(31.0, 40.0]",2
2,3,1,3,"Heikkinen, Miss. Laina",0,26,0,0,STON/O2. 3101282,7.9250,NaN,S,22,0,1,1,"(25.0, 31.0]",1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35,1,0,113803,53.1000,C123,S,44,1,2,0,"(31.0, 40.0]",2
4,5,0,3,"Allen, Mr. William Henry",1,35,0,0,373450,8.0500,NaN,S,24,0,1,1,"(31.0, 40.0]",0


In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_len,Has_Cabin,FamilySize,IsAlone,Title
0,892,3,"Kelly, Mr. James",1,2,0,0,330911,0,NaN,2,16,0,1,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,2,1,0,363272,0,NaN,0,32,0,2,0,2
2,894,2,"Myles, Mr. Thomas Francis",1,3,0,0,240276,1,NaN,2,25,0,1,1,0
3,895,3,"Wirz, Mr. Albert",1,1,0,0,315154,1,NaN,0,16,0,1,1,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,1,1,1,3101298,1,NaN,0,44,0,3,0,2


In [ ]:
drop_elements = ['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked']

train=train.drop(drop_elements, axis=1)
train.drop(['CategoricalAge'], axis=1, inplace=True)

test=test.drop(drop_elements, axis=1)

visualisations

corr

In [ ]:
# colormap=plt.cm.RdBu
# plt.figure(figsize=(15,10))
# plt.title('Pearson Correlation of Features', y=1.05, size=15)
# sns.heatmap(train.astype(float).corr(), linewidths=0.1, vmax=1.0, square=1, cmap=colormap, annot=True, fmt='.2f', linecolor='white')

PairPlot

In [ ]:
# g=sns.pairplot(train[[u'Survived',u'Pclass',u'Sex',u'Age',u'FamilySize',u'Title']], hue='Survived', palette='muted', size=1.2, diag_kind='kde', diag_kws=dict(shade=1), plot_kws=dict(s=10))
# g.set(xticklabels=[])

ensembling and stacking

In [ ]:
n_train=train.shape[0]
n_test=test.shape[0]

seed=0
n_splits=5

kf=KFold(random_state=None, shuffle=False)
kf.get_n_splits(n_splits)

#class to extend the sklearn classifier

class SKlearnHelper(object) :
  def __init__(self, clf, seed=seed, params=None) :
    params['random_state']=seed
    self.clf=clf(**params)

  def train(self, x_train, y_train) :
    self.clf.fit(x_train,y_train)

  def predict(self,x) :
    return self.clf.predict(x)
  def fit(self, x,y) :
    return self.clf.fit(x,y)
  
  def feature_importances(self,x,y) :
    print(self.clf.fit(x,y).feature_importances_)
#class to extend XGboost classifier

out of fold predictions

there are different between type kf and kf.split(train) when you use enumerate().
i think that code in kaggle causes an error because it uses old version of python and sklearn. 

In [ ]:
print(type(kf))

print(type (kf.split(train)))

print(type ( enumerate(kf.split(train))) )


<class 'sklearn.model_selection._split.KFold'>
<class 'generator'>
<class 'enumerate'>


In [ ]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((n_train,))
    oof_test = np.zeros((n_test,))
    oof_test_skf = np.empty((n_splits, n_test))

    for i, (train_index, test_index) in enumerate(kf.split(train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i , : ] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [ ]:
np.empty((n_splits, n_test))

array([[6.01239156e-316, 1.86437382e+065, 5.71088845e+287, ...,
        2.86530740e+161, 9.21468280e+242, 2.86530699e+161],
       [9.21468280e+242, 1.13288086e+277, 9.25077072e+135, ...,
        1.13824421e-119, 4.81412730e+199, 7.28413778e+174],
       [3.44442352e+175, 2.17147484e+214, 7.28413778e+174, ...,
        1.03534879e+136, 1.06396531e+224, 5.29457905e+199],
       [3.63848187e-143, 2.47724135e+214, 4.01162463e+246, ...,
        5.03889026e+175, 1.02442056e+136, 3.64550276e-143],
       [2.20891270e-094, 3.63850541e-143, 9.33876795e+252, ...,
        3.60982300e-143, 3.94357133e+180, 1.15874965e-028]])

parameters

In [ ]:
#1. rf

rf_params = {
    'n_jobs' : -1,
    'n_estimators' :100 ,
    'warm_start' : True,
    'max_depth' :6,
    'min_samples_leaf' : 2,
    'max_features' : 'sqrt',
    'verbose' :0

}

#2. ExtC

ExtC_params = {
    'n_estimators' : 100,
    'n_jobs' : -1,
    'max_depth' : 8,
    'min_samples_leaf' : 2,
    'verbose' : 0
}

#3. AdaBoost
ada_params = {
    'n_estimators' : 100,
    'learning_rate' : 0.2
}

#4. GBC
gb_params = {
    'n_estimators' : 100,
    'max_features' : 0.2,
    'min_samples_leaf' : 2,
    'verbose' : 0
    }

#5. SVM

svc_params= {
    'kernel' : 'linear',
    'C' : 0.1,
    'gamma' : 0.1
}

In [ ]:
rf = SKlearnHelper(clf=RandomForestClassifier,seed=seed, params=rf_params)
ExtC=SKlearnHelper(clf=ExtraTreesClassifier, seed=seed, params=ExtC_params)
ada=SKlearnHelper(clf=AdaBoostClassifier, seed=seed, params=ada_params)
gb=SKlearnHelper(clf=GradientBoostingClassifier, seed=seed, params=gb_params)
svc=SKlearnHelper(clf=SVC, seed=seed, params=svc_params)


In [ ]:
ExtC

Creating numpy arrays out of out train and test sets

In [ ]:
y_train=train['Survived'].ravel()

train=train.drop(['Survived'],axis=1)

x_train=train.values
x_test=test.values

In [ ]:
y_train.shape

(891,)

In [ ]:
x_train.shape

(891, 9)

Output of the First level Predictions

In [ ]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(ExtC, x_train, y_train, x_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

Training is complete


Feature importances generated from the different classifers

In [ ]:
rf_feature = rf.feature_importances(x_train, y_train)
ExtC_feature = ExtC.feature_importances(x_train, y_train)
ada_feature = ada.feature_importances(x_train, y_train)
gb_feature = gb.feature_importances(x_train, y_train)

[0.11085818 0.21113734 0.08635276 0.15060596 0.08622244 0.04680503
 0.06183271 0.00988888 0.23629671]
[0.1250037  0.43763923 0.03777764 0.05156643 0.04030206 0.08331992
 0.05528705 0.02185309 0.14725088]
[0.05 0.04 0.22 0.24 0.08 0.03 0.15 0.01 0.18]
[0.08801579 0.24966174 0.09295158 0.12061916 0.09978447 0.06362934
 0.08387923 0.0378731  0.16358559]


In [ ]:
rf_features = [0.11085818, 0.21113734, 0.08635276, 0.15060596 ,0.08622244, 0.04680503,
 0.06183271, 0.00988888 ,0.23629671]
ExtC_features = [0.1250037,  0.43763923 ,0.03777764, 0.05156643 ,0.04030206 ,0.08331992,
 0.05528705, 0.02185309, 0.14725088]
ada_features = [0.05, 0.04, 0.22, 0.24, 0.08 ,0.03 ,0.15 ,0.01 ,0.18]
gb_features = [0.08801579 ,0.24966174, 0.09295158 ,0.12061916, 0.09978447, 0.06362934,
 0.08387923, 0.0378731 ,0.16358559]

Create a df from the lists containing the feature importace data for easy plotting via the Plotly Package

In [ ]:
cols= train.columns.values

#create a df with feature
feature_df=pd.DataFrame( {'Features' : cols,
                          'RandomForest Feature importances' : rf_features,
                          'Extra Trees Feature importances' : ExtC_features,
                          'Gradient Boosting Feature importances' : gb_features,
                          'AdaBoost Feature importances' : ada_features
                          }
                        )

In [ ]:
feature_df

,Features,RandomForest Feature importances,Extra Trees Feature importances,Gradient Boosting Feature importances,AdaBoost Feature importances
0,Pclass,0.110858,0.125004,0.088016,0.05
1,Sex,0.211137,0.437639,0.249662,0.04
2,Age,0.086353,0.037778,0.092952,0.22
3,Fare,0.150606,0.051566,0.120619,0.24
4,Name_len,0.086222,0.040302,0.099784,0.08
5,Has_Cabin,0.046805,0.083320,0.063629,0.03
6,FamilySize,0.061833,0.055287,0.083879,0.15
7,IsAlone,0.009889,0.021853,0.037873,0.01
8,Title,0.236297,0.147251,0.163586,0.18


Interactive feature importances via Plotly scatterplots

go=plotly.graph_objs

In [ ]:
#Scatter plot of rf
trace=go.Scatter(
    x=feature_df['Features'].values,
    y=feature_df['RandomForest Feature importances'].values,
    mode='markers',
    marker=dict(
        sizemode='diameter',
        size = 25,
        sizeref = 1,
        # color = np.random.randn(500),
        color = feature_df['RandomForest Feature importances'].values,
        colorscale = 'Portland',
        showscale=True
                ),
    text=feature_df['Features'].values
)
data=[trace]


#layout of rf
layout =go.Layout(
    autosize= True,
    title='Random Forest Feature importance',
    hovermode='closest',
    xaxis=dict(
        title='Pop',
        ticklen=5,
        zeroline=False,
        gridwidth=2
    ),
    yaxis=dict(
        title='Feature Importance',
        ticklen=5,
        gridwidth=2
    ),
    showlegend=False
)

fig=go.Figure(data=data, layout=layout)
py.iplot(fig, filename='scatter2010')


# Scatter plot of ExtC

trace = go.Scatter(
    x=feature_df['Features'].values,
    y=feature_df['Extra Trees Feature importances'].values,
    mode = 'markers',
    marker = dict(
        sizemode = 'diameter',
        size = 25,
        sizeref = 1,
        color = feature_df['Extra Trees Feature importances'].values,
        colorscale = 'Portland',
        showscale = True
    ),
    text = feature_df['Features'].values
)
data=[trace]

layout=go.Layout(
    autosize=True,
    title ='Extra Trees Feature Importance',
    hovermode ='closest',
    yaxis=dict(
        title='Feature importance',
        ticklen=5,
        gridwidth=2
    ),
    showlegend=False
)
fig=go.Figure(data=data, layout=layout)
py.iplot(fig, filename='scatter2010')

# Scatter plot of Gradient boosting
trace = go.Scatter(
    y = feature_df['Gradient Boosting Feature importances'].values,
    x = feature_df['Features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_df['Gradient Boosting Feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_df['Features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Feature importances',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot of Ada Boosting
trace = go.Scatter(
    y = feature_df['AdaBoost Feature importances'].values,
    x = feature_df ['Features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_df['AdaBoost Feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_df['Features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'AdaBoost Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

Now let us calcurate the mean of all the feature importances and store it as a new column in the feature importance df.

In [ ]:
#create the new column containing the average of values

feature_df['mean'] = feature_df.mean(axis=1)
feature_df.head(3)

,Features,RandomForest Feature importances,Extra Trees Feature importances,Gradient Boosting Feature importances,AdaBoost Feature importances,mean
0,Pclass,0.110858,0.125004,0.088016,0.05,0.093469
1,Sex,0.211137,0.437639,0.249662,0.04,0.234610
2,Age,0.086353,0.037778,0.092952,0.22,0.109270


#Output

First level output as new features

In [ ]:
base_predictions_train = pd.DataFrame( {'RandomForest' : rf_oof_train.ravel(),
                                      'ExtraTrees' : et_oof_train.ravel(),
                                      'AdaBoost' : ada_oof_train.ravel(),
                                      'GradientBoost' : gb_oof_train.ravel()
                                      })

base_predictions_train.head()

,RandomForest,ExtraTrees,AdaBoost,GradientBoost
0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0


Correlation heatmaps of the Second Level Training set

In [ ]:
data=[
    go.Heatmap(
        z=base_predictions_train.astype(float).corr().values,
        x=base_predictions_train.columns.values,
        y=base_predictions_train.columns.values,
        colorscale='Viridis',
        showscale=True,
        reversescale = True
    )
]
py.iplot(data, filename='labelled-heatmap')

In [ ]:
x_train=np.concatenate( (et_oof_train, rf_oof_train, gb_oof_train, ada_oof_train, svc_oof_train), axis=1)
x_test=np.concatenate( (et_oof_test, rf_oof_test, gb_oof_test, ada_oof_test, svc_oof_test), axis=1)

Second level learning model via XGBoost

In [ ]:
gbm= xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimaotrs=100,
    max_depth=4,
    min_child_weight=2,
    gamma=1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    nthread=-1,
    scale_pos_weight=1
    ).fit(x_train,y_train)

gbm_predict=gbm.predict(x_test)

Producing the Submission file

In [ ]:
StackingSubmission = pd.DataFrame( { 'PassengerId' : PassengerId, 'Survived' : gbm_predict})

StackingSubmission.to_csv('StackingSubmission.csv', index=False)